In [1]:
import random
import networkx as nx
import itertools
import time

In [2]:
def is_resolving_set(B, G):
    distances = {}
    for v in G.nodes:
        distances[v] = [nx.shortest_path_length(G, v, u) for u in B]
    
    for v1, v2 in itertools.combinations(G.nodes, 2):
        if distances[v1] == distances[v2]:
            return False
    return True

In [3]:
def load_graph(file_path):
    G = nx.Graph() 

    with open(file_path, "r") as file:
        for line in file:
            parts = line.split()
            
            node1 = int(parts[0]) 
            node2 = int(parts[1])  
            G.add_edge(node1, node2) 
    return G

In [4]:
def load_gcol(file_path):
    G = nx.Graph() 

    with open(file_path, "r") as file:
        for line in file:
            parts = line.split()
            if parts[0] == 'e':
                node1 = int(parts[1]) 
                node2 = int(parts[2])  
                G.add_edge(node1, node2) 
    return G

In [5]:
def generate_population(size,G):
    return [[random.randint(0, 1) for _ in range(len(G.nodes))] for _ in range(size)]

In [6]:
def fitness(individual, G, shortest_paths):
    resolving_set = {i for i, bit in enumerate(individual) if bit == 1}

    if not resolving_set:
        return float("inf")

    unresolved_pairs = 0
    for u, v in itertools.combinations(G.nodes, 2):
        if not any(abs(shortest_paths[u][j] - shortest_paths[v][j]) > 0 for j in resolving_set if j in shortest_paths[u]):
            unresolved_pairs += 1  # Par (u, v) nije razrešen

    return len(resolving_set) + 100 * unresolved_pairs  # Penalizacija nerešenih parova

In [7]:
def tournament_selection(population, G, k=3):
    selected = random.sample(population, k)
    return min(selected, key=lambda ind: fitness(ind, G))

In [ ]:
def weighted_selection(population, G,shortest_paths):
    fitness_values = [1 / (1 + fitness(ind, G,shortest_paths)) for ind in population]
    return random.choices(population, weights=fitness_values, k=2) 

In [9]:
def crossover(parent1, parent2,G ):
    point = random.randint(1, len(G.nodes) - 1)
    return parent1[:point] + parent2[point:], parent2[:point] + parent1[point:]

In [10]:
def mutate(individual, mutation_rate=0.05):
    new_individual = individual[:]
    for i in range(len(new_individual)):
        if new_individual[i] == 1 and random.random() < mutation_rate:
            new_individual[i] = 0 
    return new_individual

In [ ]:
def genetic_algorithm(G, shortest_paths,stop,generations, pop_size=30, crossover_rate=0.5, mutation_rate=0.1):
    population = generate_population(pop_size, G)

    global_best_solution = None
    global_best_fitness = float('inf')

    for gen in range(generations):
        new_population = []

        for _ in range(pop_size // 2):
            parent1, parent2 = weighted_selection(population, G, shortest_paths)

            # Ukrštanje
            if random.random() < crossover_rate:
                child1, child2 = crossover(parent1, parent2, G)
            else:
                child1, child2 = parent1[:], parent2[:]

            # Mutacija
            child1 = mutate(child1, mutation_rate)
            child2 = mutate(child2, mutation_rate)

            new_population.extend([child1, child2])

        population = new_population

        # Pronalazak najbolje jedinke u ovoj generaciji
        best_solution = min(population, key=lambda ind: fitness(ind, G, shortest_paths))
        best_fitness = fitness(best_solution, G, shortest_paths)

        print(f"Generacija {gen + 1}, Najbolji fitnes: {best_fitness}")

        # Ažuriranje globalno najboljeg rješenja
        if best_fitness < global_best_fitness:
            global_best_fitness = best_fitness
            global_best_solution = best_solution[:]
            no_improvement = 0
        else:
            no_improvement += 1

        if no_improvement >= stop:
            print(f"Prekid: nema poboljšanja tokom {stop} generacija.")
            break

    optimal_set = {i for i, bit in enumerate(global_best_solution) if bit == 1}
    print("\nOptimalni rješavajući skup:", optimal_set, len(optimal_set))
    return optimal_set

In [12]:
G = load_gcol('grafovi\gcol\gcol26.txt')
if not nx.is_connected(G):
    print("Graf nije povezan!")
    largest_cc = max(nx.connected_components(G), key=len)
    G = G.subgraph(largest_cc).copy()

start_time1 = time.time()
shortest_paths = dict(nx.all_pairs_shortest_path_length(G))
rez = genetic_algorithm(G,shortest_paths,3,500)
end_time1 = time.time()
print("Vrijeme ", end_time1-start_time1)
print(is_resolving_set(rez,G))

<>:1: SyntaxWarning: invalid escape sequence '\g'
<>:1: SyntaxWarning: invalid escape sequence '\g'
C:\Users\Korisnik\AppData\Local\Temp\ipykernel_389688\1370227836.py:1: SyntaxWarning: invalid escape sequence '\g'
  G = load_gcol('grafovi\gcol\gcol26.txt')


Generacija 1, Najbolji fitnes: 112
Generacija 2, Najbolji fitnes: 110
Generacija 3, Najbolji fitnes: 99
Generacija 4, Najbolji fitnes: 87
Generacija 5, Najbolji fitnes: 73
Generacija 6, Najbolji fitnes: 64
Generacija 7, Najbolji fitnes: 57
Generacija 8, Najbolji fitnes: 49
Generacija 9, Najbolji fitnes: 42
Generacija 10, Najbolji fitnes: 39
Generacija 11, Najbolji fitnes: 33
Generacija 12, Najbolji fitnes: 31
Generacija 13, Najbolji fitnes: 27
Generacija 14, Najbolji fitnes: 24
Generacija 15, Najbolji fitnes: 22
Generacija 16, Najbolji fitnes: 18
Generacija 17, Najbolji fitnes: 17
Generacija 18, Najbolji fitnes: 16
Generacija 19, Najbolji fitnes: 15
Generacija 20, Najbolji fitnes: 15
Generacija 21, Najbolji fitnes: 15
Generacija 22, Najbolji fitnes: 14
Generacija 23, Najbolji fitnes: 14


C:\Users\Korisnik\AppData\Local\Temp\ipykernel_389688\1370227836.py:1: SyntaxWarning: invalid escape sequence '\g'
  G = load_gcol('grafovi\gcol\gcol26.txt')


KeyboardInterrupt: 